# Climate Anxiety in Youth and Perception of Government
## Question/Hypothesis: 
**Response Variable:** Q1 - I am worried that climate change threatens people and the planet.

**Predictors:** language, country, region, age, sex, Q2-Q8

## Importing Data

In [1]:
# Data Processing
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_graphviz
from scipy.stats import randint
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


In [ ]:
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_spss('Climate Anxiety-3.SAV')
df.head()

## Data Exploration

In [ ]:
df.columns, len(df.columns)

In [ ]:
df.dtypes

In [ ]:
df['Q1'].value_counts()

In [ ]:
df['Q1'].value_counts(normalize=True)

In [ ]:
# Count nan or 'Prefer not to say' values in the Q1 column
df['Q1'].isna().sum(), df['Q1'].str.contains('Prefer not to say').sum()

In [ ]:
df[df['Q1'].isna()]

In [ ]:
df['AgeGender'].unique(), len(df['AgeGender'])

In [ ]:
# Count the number of men and women in the dataset
df['AgeGender'].value_counts()

## Data Cleaning

#### Replace columns with more descriptive names
The original convention used to describe the columns is not very descriptive. We will replace the column names with more descriptive names.

The new convention will be the Question number (Q1, Q2, Q3, etc.) followed by a descriptive word from the poll associated with that question/subquestion.

For example, Q2 is "Does climate change make you feel any of the following?" with sad being one of the options, and the responses being yes, no, and prefer not to say. The new column name will be Q2_sad and will contain the values yes, not, or nan.

In [ ]:
columns = ['Repondent_Serial', 'language', 'country', 'age', 'sex', 'country:region', 
           'Q1', 
           'Q2_sad', 'Q2_helpless', 'Q2_anxious', 'Q2_afraid', 'Q2_optimistic',
           'Q2_angry', 'Q2_guilty', 'Q2_ashamed', 'Q2_hurt', 'Q2_depressed', 'Q2_despair',
           'Q2_grief', 'Q2_powerless', 'Q2_indifferent', 
           'Q3', 
           'Q4_hesitant', 'Q4_doomed','Q4_frightening', 'Q4_oppotunities', 'Q4_threatened', 'Q4_destroyed', 'Q4_failed',
           'Q5', 
           'Q6', 
           'Q7_concerns', 'Q7_catastrophe', 'Q7_distress', 'Q7_science', 'Q7_generations', 'Q7_trusted', 'Q7_effectiveness', 'Q7_failing', 'Q7_betraying',
           'Q8_anguished', 'Q8_abandoned', 'Q8_afraid', 'Q8_hopeful', 'Q8_angry', 'Q8_valued', 'Q8_ashamed', 'Q8_belittled', 'Q8_protected',
           'yyyymmdd','AgeGender', 
           'regionAustralia', 'regionUS', 'regionUK', 'regionIndia', 'regionNigeria', 'regionPhilippines', 'regionFinland', 'regionPortugal', 'regionBrazil', 'regionFrance', 'weight']

In [ ]:
df.columns = columns
df.columns, len(df.columns)

In [ ]:
# Drop all of the region columns
# Make a new region column that splits the country:region column by ':'
df['region'] = df['country:region'].str.split(':').str[1]
df = df.drop(columns=['regionAustralia', 'regionUS', 'regionUK', 'regionIndia', 'regionNigeria', 'regionPhilippines', 'regionFinland', 'regionPortugal', 'regionBrazil', 'regionFrance'])
df.columns, len(df.columns)

##### Note: Drop all rows with missing response variable values

In [ ]:
df = df.dropna(subset=['Q1'])

In [ ]:
order = ['extremely', 'very', 'moderately', 'a little', 'not worried']
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6))
# Sort by the value counts of the Q1 column
# sns.countplot(x='Q1', data=df, hue='Q1', palette='viridis', legend=False, order=df['Q1'].value_counts().sort_values(ascending=False).index)
sns.countplot(x='Q1', data=df, hue='Q1', palette='viridis', legend=False, order=order)
plt.xticks(rotation=45)
plt.title('Q1')
plt.show()

In [ ]:
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6))
sns.countplot(x='region', data=df, hue='region', palette='viridis', legend=False)
plt.xticks(rotation=90)
plt.title('Region')
plt.show()

In [ ]:
# Find all rows with no null values
df[df.notnull().any(axis=1)]

In [ ]:
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6))

# Limit the number of unique values in the 'sex' column to a manageable number
top_sex = df['sex'].value_counts().nlargest(10).index
df_limited = df[df['sex'].isin(top_sex)]

sns.countplot(x='sex', data=df_limited, hue='sex', palette='viridis', legend=False)
plt.xticks(rotation=45)
plt.title('sex')

for i, sex in enumerate(df_limited['sex'].unique()):
    sex_count = df_limited['sex'].value_counts()[sex]
    plt.text(i, sex_count, f'{sex_count}', ha='center')

plt.show()

##### Note: Show most populus regions in the dataset

In [ ]:
sns.set_theme(style='whitegrid')
plt.figure(figsize=(10, 6))

# Limit the number of unique values in the 'region' column to a manageable number
top_region = df['region'].value_counts().nlargest(10).index
df_limited = df[df['region'].isin(top_region)]

sns.countplot(x='region', data=df_limited, hue='region', palette='viridis', legend=False)
plt.xticks(rotation=90)
plt.title('region')

for i, region in enumerate(df_limited['region'].unique()):
    region_count = df_limited['region'].value_counts()[region]
    plt.text(i, region_count, f'{region_count}', ha='center')

plt.show()

In [ ]:
# drop unused variables 
X = df.drop(columns=['Repondent_Serial', 'language', 'country', 'country:region', 'yyyymmdd', 'AgeGender', 'region', 'weight']) # not the same as the X used in data analysis

for column in X.columns:
    summary = X[column].describe()

    print(f'{column}') 
    print(summary)

In [ ]:
# Change the sex column to be binary (0 or 1)
X['sex'] = [0 if sex == 'man' else 1 for sex in df['sex']]

#change yes/no questions to be binary (0 = no, 1 = yes?)
for column in X.columns:
    if X[column].unique().size == 2:
        X[column] = [0 if value == X[column].unique()[0] else 1 for value in X[column]]
    else:
        X[column] = X[column].astype('category').cat.codes

In [ ]:
sns.set_theme(style='whitegrid')

for column in X.columns:
    plt.figure(figsize=(10, 6))
    ax = sns.countplot(x=column, data=df, hue=column, palette='viridis', legend=False)
    plt.xticks(rotation=90)
    plt.title(f'Count Plot of {column}')

    for p in ax.patches:
        height = p.get_height()
        if not pd.isna(height):  
            ax.text(
                x=p.get_x() + p.get_width() / 2,  
                y=height,  
                s=int(height),  
                ha='center', 
                va='bottom'   
            )
    
    plt.tight_layout() 
    plt.show()

In [ ]:
# Correlations

correlations = X.corr()
q1_correlations = correlations['Q1']
print(q1_correlations.sort_values(ascending=True))

# Individually, none of the variables that we are using seem to have any practically significant correlations with Q1. 
# The variable with the Q1 correlation with the largest magnitude is Q2_afraid, with a correlaiton of .082.

## Data Analysis
Try data anaylsis with different models. Try with and without non-numeric columns (like language, country).

### Models without non-numeric columns:
#### Variables + Train and Test Split

In [ ]:
X = df.drop(columns=['Repondent_Serial', 'language', 'country', 'country:region', 'yyyymmdd', 'AgeGender', 'region', 'Q1'])
y = df['Q1']

In [ ]:
before = X['sex'].value_counts()

In [ ]:
# Change the sex column to be binary (0 or 1)
X['sex'] = [0 if sex == 'man' else 1 for sex in df['sex']]

#change yes/no questions to be binary (0 = no, 1 = yes?)
for column in X.columns:
    if X[column].unique().size == 2:
        X[column] = [0 if value == X[column].unique()[0] else 1 for value in X[column]]
    else:
        X[column] = X[column].astype('category').cat.codes

X

In [ ]:
after = X['sex'].value_counts()
before.values == after.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
category_mapping = {
    'extremely': 4,
    'very': 3,
    'moderately': 2,
    'a little': 1,
    'not worried': 0
}


In [ ]:
y_train = y_train.map(category_mapping)

In [ ]:
y_test = y_test.map(category_mapping)

In [ ]:
# All columns that start with Q6 or Q8 will be in df_ordinal
X_ordinal = X.filter(regex='Q6|Q8')
X_ordinal.columns, len(X_ordinal.columns)

(Index(['Q6', 'Q8_anguished', 'Q8_abandoned', 'Q8_afraid', 'Q8_hopeful',
        'Q8_angry', 'Q8_valued', 'Q8_ashamed', 'Q8_belittled', 'Q8_protected'],
       dtype='object'),
 10)

In [ ]:
X_nominal = X.filter(regex='Q2|Q3|Q4|Q5|Q7')
X_nominal.columns, len(X_nominal.columns)

(Index(['Q2_sad', 'Q2_helpless', 'Q2_anxious', 'Q2_afraid', 'Q2_optimistic',
        'Q2_angry', 'Q2_guilty', 'Q2_ashamed', 'Q2_hurt', 'Q2_depressed',
        'Q2_despair', 'Q2_grief', 'Q2_powerless', 'Q2_indifferent', 'Q3',
        'Q4_hesitant', 'Q4_doomed', 'Q4_frightening', 'Q4_oppotunities',
        'Q4_threatened', 'Q4_destroyed', 'Q4_failed', 'Q5', 'Q7_concerns',
        'Q7_catastrophe', 'Q7_distress', 'Q7_science', 'Q7_generations',
        'Q7_trusted', 'Q7_effectiveness', 'Q7_failing', 'Q7_betraying'],
       dtype='object'),
 32)

#### Model 1: Oridnal/Nominal Logistic Regression

#### Model 2: Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
y_pred_RF = rf.predict(X_test)
accuracy_score(y_test, y_pred_RF)

In [ ]:
cm = confusion_matrix(y_test, y_pred_RF)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
# Export the first three decision trees from the forest

for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, y_train)

In [ ]:
# best model!
best_rf = rand_search.best_estimator_

print('Best hyperparameters:',  rand_search.best_params_)

ConfusionMatrixDisplay(confusion_matrix=cm).plot()

In [ ]:
print(np.mean(y_pred == y_test), np.mean(y_pred != y_test))

##### Random Forest (Bagging Classifier)

In [ ]:
tree = DecisionTreeClassifier(min_samples_leaf=7) #min_samples_split=11

In [ ]:
ens_model = BaggingClassifier(estimator=tree, n_estimators=100, 
                                  bootstrap=True,
                                  bootstrap_features=True,     # RF
                                  oob_score=True,                     
                                  random_state=0).fit(X_train, y_train)

print('oob score =', ens_model.oob_score_)

pred = ens_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, pred)
ConfusionMatrixDisplay(confusion_matrix=cm).plot();

In [ ]:
accuracy_score(y_test, pred)

In [ ]:
precision_score(y_test, pred, average='macro')

In [ ]:
recall_score(y_test, pred, average='macro')

#### Model 3: Gradient Boosting

In [ ]:
gbc = GradientBoostingClassifier()
 
gbc.fit(X_train, y_train)
 
y_pred_gbc = gbc.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred_gbc)

In [ ]:
cm_gbc = confusion_matrix(y_test, y_pred_gbc)
ConfusionMatrixDisplay(confusion_matrix=cm_gbc).plot();

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gbc))

In [ ]:

feature_importances_gbc = gbc.feature_importances_
indices_gbc = np.argsort(feature_importances_gbc)[::-1]

feature_names = df.columns
top_feature_names = feature_names[indices_gbc[:3]]
top_feature_importances = feature_importances_gbc[indices_gbc[:3]]

plt.figure(figsize=(10, 6))
plt.title("Top 5 Feature Importances")
plt.barh(range(top_n), top_feature_importances[::-1], align="center")  
plt.yticks(range(top_n), top_feature_names[::-1])  
plt.xlabel("Feature Importance")
plt.show()


In [ ]:
# Hyperparameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

# GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=GradientBoostingClassifier(random_state=42), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best Parameters
print("Best Hyperparameters:", grid_search.best_params_)

# Train the model with the best parameters
best_gb_clf = grid_search.best_estimator_
y_pred_best = best_gb_clf.predict(X_test)

# Evaluate the best model
print(f'Best Model Accuracy: {accuracy_score(y_test, y_pred_best):.4f}')
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best))


In [ ]:

cv_scores = cross_val_score(gbc, X, y, cv=5, scoring='accuracy')
print(f'Cross-validation Accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}')


#### Model 4: Neural Network